In [34]:
import pandas as pd
import geopandas as gpd

In [37]:
unique_kalpiot = pd.read_csv('unique_kalpiot_basic_google_geocoding.csv', index_col=0)
stats = gpd.read_file('data/ezorim_statistiim_2022.gdb')

In [ ]:
# merge statistical areas

In [30]:
# Missing Lat
unique_kalpiot.loc[unique_kalpiot['Latitude'].isna()].head()

,town_symbol,town_name,location_symbol,ballot_address,ballot_location,bldg_nubmer,street,geocode_address,Latitude,Longitude
1742,146,דבורה,1,דבורה,מזכירות,NaN,דבורה,"דבורה, דבורה",NaN,NaN
1781,1110,חבר,1,חבר,מזכירות,NaN,חבר,"חבר, חבר",NaN,NaN
1973,1413,מחנה יהודית,1,מחנה יהודית,מועדון חוגים,NaN,מחנה יהודית,"מחנה יהודית, מחנה יהודית",NaN,NaN
2186,526,סולם,1,סולם,"בי""ס יסודי ממלכתי",NaN,סולם,"סולם, סולם",NaN,NaN
2452,1208,הילה,1,הילה,מזכירות,NaN,הילה,"הילה, הילה",NaN,NaN


In [31]:
i = 1742
unique_kalpiot.loc[i, 'Latitude']  = 32.55307
unique_kalpiot.loc[i, 'Longitude'] = 35.26391


In [32]:
unique_kalpiot.loc[unique_kalpiot['Latitude'].isna()]

,town_symbol,town_name,location_symbol,ballot_address,ballot_location,bldg_nubmer,street,geocode_address,Latitude,Longitude
1781,1110,חבר,1,חבר,מזכירות,NaN,חבר,"חבר, חבר",NaN,NaN
1973,1413,מחנה יהודית,1,מחנה יהודית,מועדון חוגים,NaN,מחנה יהודית,"מחנה יהודית, מחנה יהודית",NaN,NaN
2186,526,סולם,1,סולם,"בי""ס יסודי ממלכתי",NaN,סולם,"סולם, סולם",NaN,NaN
2452,1208,הילה,1,הילה,מזכירות,NaN,הילה,"הילה, הילה",NaN,NaN
4071,3717,חרמש,1,חרמש,מזכירות,NaN,חרמש,"חרמש, חרמש",NaN,NaN
6060,848,בית רבן,1,בית רבן,מכללת גבעת וושינגטון,NaN,בית רבן,"בית רבן, בית רבן",NaN,NaN
6247,1411,מחנה הילה,1,מחנה הילה,מועדון משפחות,NaN,מחנה הילה,"מחנה הילה, מחנה הילה",NaN,NaN
6248,1412,מחנה תל נוף,1,מחנה תל נוף,מועדון משפחות,NaN,מחנה תל נוף,"מחנה תל נוף, מחנה תל נוף",NaN,NaN
9433,44,זוהר,1,זוהר,מועדון זוהר,NaN,זוהר,"זוהר, זוהר",NaN,NaN
9462,1414,מחנה מרים,1,מחנה מרים,מועדון קצינים,NaN,מחנה מרים,"מחנה מרים, מחנה מרים",NaN,NaN


In [29]:
len(unique_kalpiot.loc[unique_kalpiot['Latitude'].isna()])

32

In [23]:
unique_kalpiot

,town_symbol,town_name,location_symbol,ballot_address,ballot_location,bldg_nubmer,street,geocode_address,Latitude,Longitude
0,3000,ירושלים,18,"מגן האלף,1",סמינר בית יעקב - עטרת חן,1,מגן האלף,"מגן האלף 1, ירושלים",31.794685,35.222309
1,3000,ירושלים,383,"שבטי ישראל,27",משרד החינוך,27,שבטי ישראל,"שבטי ישראל 27, ירושלים",31.783704,35.224509
3,3000,ירושלים,5,"מגן האלף,3","מרכז קהילתי ""אשכולות""",3,מגן האלף,"מגן האלף 3, ירושלים",31.795200,35.223370
11,3000,ירושלים,7,"הנביאים,42","בי""ס אורט אולייסקי",42,הנביאים,"הנביאים 42, ירושלים",31.783929,35.222735
14,3000,ירושלים,21,"אביגדורי,4","בי""ס בנות - דחסידי באיאן רוזין",4,אביגדורי,"אביגדורי 4, ירושלים",31.785457,35.219290
...,...,...,...,...,...,...,...,...,...,...
11685,8800,שפרעם,9,ג'בור יוסף ג'בור,"בי""ס ממלכתי ו' (אל מכתב) ג'בור",NaN,ג'בור יוסף ג'בור,"ג'בור יוסף ג'בור, שפרעם",32.806137,35.169241
11689,8800,שפרעם,11,שפרעם,"בי""ס מקיף עירוני א'-סאלח חניפס",NaN,שפרעם,"שפרעם, שפרעם",32.805765,35.169971
11693,8800,שפרעם,20,"מרכז מסחר-ע עתיקה,1","בי""ס קתולי חדש",1,מרכז מסחר-ע עתיקה,"מרכז מסחר-ע עתיקה 1, שפרעם",32.805765,35.169971
11699,8800,שפרעם,17,שפרעם,"בי""ס אלבורג'",NaN,שפרעם,"שפרעם, שפרעם",32.805765,35.169971


In [33]:
unique_kalpiot.to_csv('unique_kalpiot_basic_google_geocoding.csv')